<center>
<h3>Agents Example(creating custom agents)</h3>
</center>

## Agents

<p>By themselves language model can't take actions- they just ouput text</p>
<p>Agents are system that use LLM as a reasoning engine to determine which action to take(means which tool should be called.) and what the input to those actions should be. The result of those action can be feed back into the agent and it determine whether more actions are needed or whether it is okay to finish.</p>

<p><code>LangGraph</code> is used to create highly controllable and cutomizable agents</p>

#### Simple agent  implementation using LangGraph

In [18]:
from langgraph.graph import START,END,StateGraph #for graph
from langchain_core.tools import BaseTool # for tools 
from pydantic import BaseModel,Field # for type
from typing import Optional,Type,Literal,TypedDict
from langchain_core.callbacks import CallbackManagerForToolRun,AsyncCallbackManagerForToolRun



In [17]:
class BaseInputType(BaseModel):
    a : int = Field("First Number")
    b: int = Field("Second Number")

class BaseOutputType(BaseModel):
    c : int = Field("Result")
    
# Tool Implementation

# Note: It's important that every field has type hints. BaseTool is a
# Pydantic class and not having type hints can lead to unexpected behavior.

class SimpleCalculator(BaseTool):
    name : str = "Simple Calculator"
    description : str = "Helpful for simple math question"
    args_schema: Type[BaseModel] = BaseInputType
    return_direct:bool = True
    
    
    # for normal calls
    def _run(self,a : int,b:int,run_manager:Optional[CallbackManagerForToolRun] = None) -> BaseOutputType:
        return a*b
    
    # If the calculation is cheap, you can just delegate to the sync implementation
    # as shown below.
    # If the sync calculation is expensive, you should delete the entire _arun method.
    # LangChain will automatically provide a better implementation that will
    # kick off the task in a thread to make sure it doesn't block other async code. 
        
    # for async calls
    async def _arun(self,a:int,b:int, run_manager:Optional[AsyncCallbackManagerForToolRun] = None)->BaseOutputType:
        return self._run(a,b,run_manager)

# checking the tool
multiply = SimpleCalculator()

In [5]:
print(multiply.name)
print(multiply.description)
print(multiply.args_schema)

Simple Calculator
Helpful for simple math question
<class '__main__.BaseInputType'>


In [15]:
print(multiply.invoke({"a":2,"b":3}))
print(await multiply.ainvoke({"a":25,"b":25}))

6
625


### Agent(using LangGraph)

In [29]:
# Type of state
class State(TypedDict):
    first_num : int
    sec_num : int
    result : int

# Defining the functionality of Node
def node_1(state):
    # let's update the state in node 1
    print("--- Inside Node 1 (Updating the state) ---- \n")
    print("State : ",state,"\n")
    return ({"first_num" : state["first_num"], "sec_num":state["sec_num"]})

def node_2(state):
    print("---- Inside Node 2 (returning multiply) ----\n")
    return({"result":state["first_num"] * state["sec_num"]})

def node_3(state):
    print("---- Inside node 3 (returning addition) ----\n")
    return({"result" : state["first_num"] + state["sec_num"]})

def deciding_node(state)-> Literal["node_2","node_3"]:
    if state["first_num"] < state["sec_num"]:
        return "node_2"
    return "node_3"

# Creating Graph
builder = StateGraph(State)

# Adding node to graph
builder.add_node("node_1",node_1)
builder.add_node("node_2",node_2)
builder.add_node("node_3",node_3)

# Adding edge between graph
builder.add_edge(START,"node_1")
builder.add_conditional_edges("node_1",deciding_node)
builder.add_edge("node_2",END)
builder.add_edge("node_3",END)

# at last compiling the graph
graph = builder.compile()

In [30]:
graph.invoke({"first_num":10,"sec_num":30})

--- Inside Node 1 (Updating the state) ---- 

State :  {'first_num': 10, 'sec_num': 30} 

---- Inside Node 2 (returning multiply) ----



{'first_num': 10, 'sec_num': 30, 'result': 300}

In [31]:
graph.invoke({"first_num":100,"sec_num":30})

--- Inside Node 1 (Updating the state) ---- 

State :  {'first_num': 100, 'sec_num': 30} 

---- Inside node 3 (returning addition) ----



{'first_num': 100, 'sec_num': 30, 'result': 130}